In [55]:
# Importing packages and loading env:
import pandas as pd
import numpy as np
import re
import json
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
load_dotenv()
from src.api import exchangerate_api_request
from src.api import battuta_request_authorized
from src.webscraping import get_soup
from src.clean import resub_list

# df_final = pd.read_csv('./input/cleaned_enriched_df.csv')

In [36]:
df_final.head()

,name,state,region,city,latitude,longitude,cuisine,stars,min_price_EUR,max_price_EUR,url
0,Kilian Stuba,Austria,Austria,Kleinwalsertal,47.348580,10.17114,Creative,1,90,120,https://guide.michelin.com/at/en/vorarlberg/kl...
1,Pfefferschiff,Austria,Austria,Hallwang,47.837870,13.07917,Classic cuisine,1,59,120,https://guide.michelin.com/at/en/salzburg-regi...
2,Esszimmer,Austria,Austria,Salzburg,47.806850,13.03409,Creative,1,45,128,https://guide.michelin.com/at/en/salzburg-regi...
3,Carpe Diem,Austria,Austria,Salzburg,47.800010,13.04006,Market cuisine,1,61,118,https://guide.michelin.com/at/en/salzburg-regi...
4,Edvard,Austria,Austria,Wien,48.216503,16.36852,Modern cuisine,1,33,158,https://guide.michelin.com/at/en/vienna/wien/r...


In [ ]:
# Web scraping https://guide.michelin.com to get services:
# names_list = []
i = 0
services_list = []
for e in df_final['url']:
    soup = get_soup(e)
    services = soup.select('.restaurant-details__services--content')
    services_rest = [re.sub('\n','',e.text) for e in services]
    services_list.append([df_final.at[i,'name'], services_rest])
    i+=1
print(services_list)

In [70]:
import json
url = 'https://api.foursquare.com/v2/venues/explore'
params = dict(
  client_id = os.getenv("FOURSQUARE_CLIENT_ID"),
  client_secret = os.getenv("FOURSQUARE_CLIENT_SECRET"),
  v='20180323', # version parameter
  # ll='40.7243,-74.0018',
  ll='{},{}'.format(df_final['latitude'][0],df_final['longitude'][0]),  
  query='coffee',
  limit=1
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [71]:
data

{'meta': {'code': 200, 'requestId': '5dc78051f9dbde0038335eb8'},
 'response': {'warning': {'text': 'There aren\'t a lot of results for "coffee." Try something more general, reset your filters, or expand the search area.'},
  'suggestedRadius': 10000,
  'headerLocation': 'Hirschegg',
  'headerFullLocation': 'Hirschegg',
  'headerLocationGranularity': 'city',
  'query': 'coffee',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 47.3574520088127,
    'lng': 10.182767183090048},
   'sw': {'lat': 47.35475234902725, 'lng': 10.185541301202163}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c4af42dbad7a593b33128ab',
       'name': 'Cantina Vertical',
       'contact': {},
       'location': {'address': 'Walserstr. 70',
        'lat': 47.356102178919976,
        'lng'

In [33]:
# df_final.to_csv('./input/cleaned_enriched_df.csv', index=False)